*Getting Started with Tensorflow 2*

*Week 3 Lecture Notes*

*Coursera/Imperial College London*

-----------------

# Validation, regularisation and callbacks

### **Validation sets**

Validation sets is useful as a measure of how well our model is performing outside of the training set. So the model never uses validation set to train on but its performance is evaluated on the validation set during training. 

```
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(128, activation='tanh'))
model.add(Dense(2))

opt = Adam(learning_rate=0.05)
model.compile(optimizer=opt, loss='mse', metrics=['mape'])

history = model.fit(inputs, targets, validation_split=0.2)
```

Remember the first dimension of the `inputs` array would be the number of examples in the training set and the remaining dimensions would be for the features. Similarly, the first dimension of the numpy array `targets` will be the same, number of samples in the dataset. The second dimension here should be equal to two, as this needs to match up with the dimension of the output layer of our network. 

Let's say then that we want to track the performance of our model on a separate held-out validation set. One way we can do this is to pass in the `validation_split` keyword argument. What this does is it automatically splita the inputs and the targets into separate training and validation sets. The 0.2 you see here means that 20 percent of the data will be held back for validation. The model's performance is then recorded on both the training and validation sets.

The history object also records the performance on the validation set in addition to the training set loss and metrics over the course of the model training. These values are stored inside the history attributes of the history object and this attribute is a Python dictionary. If we were to take a look at this dictionary, we would see that it now not only has keys for the training set loss and metric values, but also for the validation loss of metric values.

`print(history.history.keys())    # dict_keys(['loss', 'mape', 'val_loss', 'val_mape'])`


Sometimes, datasets have already been packaged up for us with the training and test split. This is the case, for example, when you load the `fashion_mnist` dataset with the `tf.keras.datasets` module. You could use this split as a ready-made training and validation split. So instead of getting the `model.fit` method to make this split for us, we explicitly give it the validation set. If we want to do that, then we can use the `validation_data` keyword argument. This argument takes a tuple of inputs and outputs, as before the model will then record the performance on this validation set during the training run.

```
import tensorflow as tf

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

model.fit(X_train, y_train, validation_data=(X_test, y_test))
```

Finally, we might also want to make a training and validations with our ourselves, before feeding them to the `model.fit` method. For example, we could use the train test split function from sklearn to make a training and validation split and then feed them into the fit method, again using the validation data keyword argument

```
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

model.fit(X_train, y_train, validation_data=(X_val, y_val))
```

So now we've seen how to include validation sets into our training room using either the validation split keyword argument or the validation data keyword argument. This is a really important feature to build into our training workflow. 

---------------
**Question:** How many keys will the history.history dictionary have after the following training run?

```
model.compile(loss=‘binary_crossentropy’, metrics=[‘accuracy’, ‘mae’])
history = model.fit(X_train, y_train, validation_data=(X_val, y_val))
```

**Answer**: Six. The complete list is `[‘loss’, ‘accuracy’, ’mae’, ‘val_loss’, ‘val_accuracy’, ’val_mae’]`

----------------

**Question**: What should `p1` and `p2` be to ensure a 60/20/20 training/validation/test split?

```
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=p1)
model.fit(X_train, y_train, validation_split=p2)
```

**Answer** `p1=0.2, p2=0.25`

----------------


###**Model Regularisation**

You can include regularization techniques into the model training that have the effect of constraining the model capacity in preventing overfitting. In particular, we're going to look at using L2 weight regularization, which is also known as weight decay in a context of neural networks, as well as L1 weight regularization and you'll see how to include Dropouts in your models. Let's start with adding in weight decay into our model.

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
                    Dense(64, activation='relu',
                          kernel_regularizer=tf.keras.regularizers.l2(0.001)),
                    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adadelta',loss='binary_crossentropy', metrics=['acc'])
 #model.fit(inputs, targets, validation_split=0.25)

Here we have a feedforward network with a single hidden layer. This is a binary classification model. As you can see, the output has just one neuron and the sigmoid activation function. We can add in weight decay when we define our model. Dense layers as well as convolutional layers have an optional `kernel_regularizer` keyword argument. To add in weights decay or L2 regularization, we set the `kernel_regularizer` argument equal to `tf.keras.regularizes.l2` object. 

This object is created with one required arguments, which is the coefficient that multiplies the sum of squared weights in this layer. Remember that a dense layer has weights and biases, and the weight matrix is sometimes called the kernel. Here we've set the weight decay coefficients to be equal to 0.001. 

The weight decay penalty term is automatically added to the loss-function when we compile the model.

So here our loss-function would be the binary cross entropy calculated using the model predictions and the true labels plus the sum of squared weights of the first layer around model multiplied by the weight decay coefficient of 0.001. This sum of squared weights term has the effect of penalizing large values of the weights, which encourages the model to find the simpler function that fits the data. This means the model is less likely to overfit to the training set. 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
                    Dense(64, activation='relu',
                          kernel_regularizer=tf.keras.regularizers.l1(0.005)),
                    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adadelta',loss='binary_crossentropy', metrics=['acc'])
 #model.fit(inputs, targets, validation_split=0.25)

Instead of L2 regularization, we could use L1 regularization. In this case, the penalty term is a sum of absolute weight values, instead of a sum of squared weight values, we can use L1 regularization in a very similar way. This time we're creating a `tf.keras.regularizers.l1` object and passing that to the kernel regularizer argument. The regularization coefficient this time is 0.005. L1 regularization has the effect of sparsifying the network weights. Or in other words, it results in some of those weights being set to zero. In fact, there's no reason why you couldn't use both L1 and L2 regularization.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
                    Dense(64, activation='relu',
                          kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.005, l2=0.001)),
                    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adadelta',loss='binary_crossentropy', metrics=['acc'])
 #model.fit(inputs, targets, validation_split=0.25)

Here we've created a `tf.keras.regularizers.l1l2` object and pass that to the kernel regularizer argument. The coefficients for the regularizers can be set independently. So now the regularizer object has got two keyword arguments, one the L1 regularization and one for the L2 regularization. 

In all of these examples so far, we've only been applying regularization to the weights matrix of the first dense layer. It's quite typical to only apply regularization to the weight matrix or the kernel of a dense or convolutional layer. However, it is perfectly possible to include regularization for the bias as well.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
                    Dense(64, activation='relu',
                          kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.005, l2=0.001),
                          bias_regularizer=tf.keras.regularizers.l2(0.001)),
                    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adadelta',loss='binary_crossentropy', metrics=['acc'])
 #model.fit(inputs, targets, validation_split=0.25)

Here we can see how to do just that. In the constructor of a dense or convolutional layer, there is also an optional bias regularizer argument. We can pass this argument a regularizer object in just the same way as we did before. Again, this will add a penalty term to the loss function, this time for the bias parameters. 

The final thing I'd like to show you is how to include Dropout in your network. Dropout also has a regularizing effect in neural networks and intensive flow, we can include Dropout as just another layer. 



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
                    Dense(64, activation='relu'),
                    Dropout(0.5),
                    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adadelta',loss='binary_crossentropy', metrics=['acc'])

 #model.fit(inputs, targets, validation_split=0.25)     # Training mode, with dropout
 #model.evaluate(val_inputs, val_targets)               # Testing mode, no dropout
 #model.predict(test_inputs)                            # Testing mode, no dropout



So here, you can see we're importing not only the dense layer as before, but also the Dropouts layer. 

Now in our sequential model, we've included the Dropouts layer in our list of layers. The Dropouts layer has a required arguments, which is the Dropouts rate. Here the rate has been set to 0.5 and that means that each weight connection between these two dense layers is set to zero with probability 0.5. (This is sometimes referred to as Bernoulli Dropout, since the weights are effectively being multiplied by a Bernoulli random variable.) Each of the weights are randomly dropped out independently from one another and Dropout has also applied independently across each element in the batch at training time. Again, we can compile and fit our model as normal, with Dropouts layers included in our model. 

Remember that when we're using Dropout, we typically have two different modes for how we run the network. At training time, we randomly Dropout weights in the way that I've described and we can think of this as Training mode. However, when we're evaluating our model or making predictions from it, we stopped randomly dropping out the weights and we can think of this as testing mode. These two modes are automatically handled behind the scenes by the `model.fit`, `model.evaluate`, and `model.predict` methods. But it's worth bearing in mind that this is happening in the Dropout layer and later on we'll look at ways of getting more lower level control over the network and you'll see then how these two modes of operation for Dropout can be controlled. 

###**Callbacks**

But even now we're monitoring the performance of our model on the validation set, we're still having to fix the number of epochs that the model is training for ahead of time. What we would like to do is to have the ability to not only monitor the performance of the network, but also perform certain actions depending on those performance measures.

That's where callbacks come in. Callbacks are an important type of object TensorFlow and Keras that are designed to be able to monitor the loss in metrics at certain points in the training run and perform some action that might depend on those loss in metric values. Let's take a look at how callbacks are constructed. 

In TensorFlow, all callbacks are stored in the `tensorflow.keras.callbacks` module. Inside that module, there's a base class called `callback` which all other callbacks inherit from. You can also subclass the callback based class yourself to create your own callbacks, and that's what we're going to start to look at first as this is a really nice way to get a feel for how these objects work. 



```

from tensorflow.keras.callbacks import Callback

class my_callback(Callback):
  
  def on_train_begin(self, logs=None):
    # Do something at the start of training
    
  def on_train_batch_begin(self, batch, logs=None):
    # Do sonething at the start of every batch iteration

  def on_epoch_end(self, epoch, logs=None):
    # Do something at the end of every epoch

history = model.fit(X_train, y_train, epochs=5, callbacks=[my_callback()])

```

Here I'm creating a new class called `my_callback`, and you can see that I'm sub-classing the callback base class. This base class has a series of methods that can be overridden in our new class definition. These methods will then be called at different points in the training. 

For example, one of the methods of the callback base class is called `on_train_begin`. This method optionally uses the `logs` keyword argument, and you'll find out more about that in a notebook reading coming up in this week. For now though, you just need to know that as the name suggests, this method will be called once at the start of training. So we can override this method in our new callback to perform some action at the beginning of the training run. 

Similarly, there are more methods of the base class that are called at other points in the training. You can pretty much just read it from the name `on_train_batch_begin` is called at the start of every batch or every iteration of the training. You can see that this method has a `batch` argument. So when this method is called, the batch number will be passed in using this argument. So if you want you can use the batch number during training to perform some action within this method. 

The train part of `on_train_begin` and `on_train_batch_begin` means that these methods will only be used in the training run. We can actually also use callbacks in evaluation and prediction runs as well.

`on_epoch_end` is the method that's called at the end of every epoch. This method has an `epoch` arguments. So similar to before, when this method is called, it gets past the epoch number, so we could use this number within the method if we want to.

Once we've defined the actions we want our callback to take at these various parts of the training run, all we need to do is to pass in the callback to `model.fit`. `model.fit` has a callbacks keyword argument, and this takes a list of callback objects. So here we're passing in an instance of our callback. Remember again that `model.fit` returns a history object. So by this point, it should make sense that this object is actually an example of a callback. It's a callback that is automatically included into every training run whenever we call `model.fit`. The action that this callback takes is simply to record the loss and metric values and store them as a dictionary in its history attribute. 

There are quite a few built-in callbacks in TensorFlow and we are going to be covering several of them in the later weeks of the course, but you can take a look at the list of built-in callbacks, if you go to the API documentation on TensorFlow.org. 

###**Early stopping and patience**

We'll use a callback to implement another regularization approach called early stopping. Early stopping is a technique that monitors the performance of the network for every epoch on a held out validation set during the training run, and terminates the training conditional on the validation performance. The Keras module contains a built-in callback designed for this purpose called the early stopping cutback. 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential([
                    Conv1D(16, 5, activation='relu', input_shape=(128, 1)),
                    MaxPooling1D(4),
                    Flatten(),
                    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, min_delta=0.01, mode='max')
#early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.01, mode='min')

#model.fit(X_train, y_train, validation_split=0.2, epochs=100, callbacks=[early_stopping])

Let's see how it works. In this example, I've built a convolutional network starting with a `Conv1D` layer, followed by a `MaxPoolinglayer`. Note that the input is two-dimensional, so this could be a univariate time series of length 128, for example. Then we've flatten the input by unrolling into one long vector, and finally, we have a dense layer with a 10 layer `Softmax`. We're compiling the model, with the `AdamOptimizer`, `categorical_crossentropy_loss`, and we're tracking the `categorical_accuracy` metric.